In [ ]:
# %load data_processing/token_price.py
import requests

def convert_blockheight_to_date(snapshot,api_key): 
    
    """Takes block_height and returns date.

    Args:
        snapshot (str): block height

    Returns:
        str: date of format YYYY-MM-DD

    Raises:
        None
    """
    
    url = f"https://api.covalenthq.com/v1/eth-mainnet/block_v2/{snapshot}/?key={api_key}"

    response = requests.get(url)
    
    date = response.json()['data']['items'][0]['signed_at']
    
    date_YMD = date.split('T')[0]
    
    return date_YMD


def usd_value_decimals_token(snapshot, contractAddress, api_key, chainName='eth-mainnet', quoteCurrency='USD'): 
    
    """Takes blockheight and contract address of token and
    returns contract_decimals and value in usd.

    Args:
        snapshot (str): block height.
        contractAddress (str): token contract address

    Returns:
        dict with keys and value pairs: 
        'contract_address': contractAddress (str)
        'token_price_usd': price_usd_per_token (int)
        'contract_decimals': decimals (int)

    Raises:
        None
    """
    

    
    
    date = convert_blockheight_to_date(snapshot, api_key)
    
    
    url = f"https://api.covalenthq.com/v1/pricing/historical_by_addresses_v2/{chainName}/{quoteCurrency}/{contractAddress}/?key={api_key}"
    
    params = {
    "from": date, # (YYYY-MM-DD)
    "to": date # (YYYY-MM-DD)
    }

    response = requests.get(url, params=params)
    
    
    try: 
        # token price 
        price_usd_per_token = response.json()['data'][0]['prices'][0]['price']
        
        # decimals
        decimals = response.json()['data'][0]['contract_decimals']
    
    except: 
        
        price_usd_per_token = np.nan
        decimals = np.nan
        
    
    return {'contract_address': contractAddress, 'token_price_usd': price_usd_per_token, 'contract_decimals': decimals}
    
    

In [4]:
### .p5 file implementation

# data  packages
import pandas as pd
import numpy as np

# data manipulation packages
import math
from scipy.stats import hypergeom 
from itertools import combinations, islice

# data processing 
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

# 
# from data_processing.token_price import convert_blockheight_to_date, usd_value_decimals_token

# os 
import os
from os.path import join
from dotenv import load_dotenv

# auxillary 
from tqdm import tqdm

# data storage
import h5py

# timestamping 
import datetime
import time 

# load environment
load_dotenv()

# specify paths
path = os.environ["PROJECT_PATH"]
covalent_api_key =  os.environ["COVALENTHQ_API_KEY"]
output_path = '/local/scratch/exported/governance-erc20/project_erc20_governance_data/wallet_projection_output/output_f-none_grouping'

# Snapshot selection
df_snapshot = pd.read_csv("../assets/snapshot_selection.csv")

# Address selection
df_addresses = pd.read_csv("../assets/df_final_token_selection_20230813.csv")

# Burner addresses (remove burner addresses)
known_burner_addresses = [
    "0x0000000000000000000000000000000000000000",
    "0x0000000000000000000000000000000000000000",
    "0x0000000000000000000000000000000000000001",
    "0x0000000000000000000000000000000000000002",
    "0x0000000000000000000000000000000000000003",
    "0x0000000000000000000000000000000000000004",
    "0x0000000000000000000000000000000000000005",
    "0x0000000000000000000000000000000000000006",
    "0x0000000000000000000000000000000000000007",
    "0x000000000000000000000000000000000000dead",
]



price_dict = {}

for snapshot in df_snapshot[df_snapshot['Block Height'] >= 11547458]['Block Height']: # 11547458
    
    
    print(f"Snapshot for Block Height: {snapshot} - {datetime.datetime.now()}")
    
    # Load data
    ddf = pd.read_csv(join(path, f"token_balance_lookup_tables_labelled/df_token_balenace_labelled_greater_01pct_bh{snapshot}_v2.csv"))
    ddf = ddf.rename(columns={'address_x': 'address'})
    ddf = ddf[ddf.value > 0]
    ddf = ddf[ddf.token_address.isin(df_addresses.address)]
    ddf = ddf[~ddf.address.isin(known_burner_addresses)]
    
    snapshot_price_dict = {} 
    
    for contract_address in ddf.token_address.unique(): 
        
        print(contract_address)

        response = usd_value_decimals_token(snapshot, contract_address, covalent_api_key)

        contract_decimals = response['contract_decimals']
        token_price_usd = response['token_price_usd']
        
        if contract_decimals != 18: 
            break

        snapshot_price_dict[contract_address] =  token_price_usd

    price_dict[snapshot] =  snapshot_price_dict
    
    time.sleep(20)

    

Snapshot for Block Height: 11659570 - 2023-10-25 14:12:17.849043


KeyboardInterrupt: 

In [15]:
df_price_table = pd.DataFrame(price_dict)

In [16]:
df_price_table.to_csv('../assets/price_table.csv')